In [2]:
import pandas as pd

In [136]:
#Import the raw mass-spectrometry data
raw_ms_data = pd.read_csv("bovine_DMT_2020_01_24_reanalysis.csv")
raw_ms_data = raw_ms_data.drop("Name",axis=1) #Drop the empty names column
raw_ms_data.head(10) #print first 5 entries to screen

,Unique,Total,reference,Gene Symbol,MWT(kDa),AVG
0,133,154,tr|A0A3Q1NE82|A0A3Q1NE82_BOVIN,AHNAK,614.26,3.1225
1,7,109,sp|Q3MHM5|TBB4B_BOVIN,TUBB4B,49.80,3.3008
2,13,101,sp|Q2T9S0|TBB3_BOVIN,TUBB3,50.40,3.3130
3,62,98,tr|A0A3Q1M0Y8|A0A3Q1M0Y8_BOVIN,CROCC2,169.52,3.1375
4,16,91,tr|F6RP72|F6RP72_BOVIN,LOC100295712; tubulin alpha chain,50.85,3.0554
5,80,88,tr|F1MMQ8|F1MMQ8_BOVIN,SPTA1,280.78,3.3721
6,71,84,tr|F1MKE9|F1MKE9_BOVIN,SPTB,268.25,3.5225
7,18,83,tr|A0A452DIL8|A0A452DIL8_BOVIN,TUBB2B,49.94,3.7333
8,27,75,tr|Q58DT9|Q58DT9_BOVIN,ACTA2,45.19,2.7932
9,68,72,tr|A0A3Q1MRS3|A0A3Q1MRS3_BOVIN,PLEC,535.22,2.9450


In [6]:
import urllib.parse
import urllib.request

def req_uniport(gene):
    #Specify the database to query - we are just quering the general protein database
    url = 'https://www.uniprot.org/uniprot/'

    #entering the parameters we care about
    params= {
        'query':gene,
        'format':'fasta'
    }

    data=urllib.parse.urlencode(params)
    data = data.encode('utf-8')
    req = urllib.request.Request(url,data)
    
    return req

def decode_uniport(req):
    with urllib.request.urlopen(req) as f:
        response = f.read()
        response = response.decode('utf-8')
    
    return response

def from_response_get_aaonly(response):
    start = int(0)

    for n_line, i in enumerate(response.split()):
        if i[:2] == "SV":
            start=n_line
            break
    return "".join(response.split()[n_line+1:])

    
def get_uniprot_aa_seq(gene):
    request=req_uniport(gene)
    response=decode_uniport(request)
    seq=from_response_get_aaonly(response)
    return seq
    

In [186]:
bovine_sequences=[]

#Iterate over all genes. Append sequences to list
for _, bovine_gene in zip(range(len(raw_ms_data)),raw_ms_data["reference"]):
    gene = bovine_gene.split("|")[2]
    bovine_sequences.append(get_uniprot_aa_seq(gene))

In [192]:
#convert list to a series and append to dataframe
raw_ms_data["Bovine_Seq"]=pd.DataFrame(bovine_sequences,columns=["Bovine_Seq"])
raw_ms_data.head(25)

,Unique,Total,reference,Gene Symbol,MWT(kDa),AVG,Bovine_Seq
0,133,154,tr|A0A3Q1NE82|A0A3Q1NE82_BOVIN,AHNAK,614.26,3.1225,MEKEEETTRELLLPNWQGSGSHGLTITQRDDGVFVQEVMQNSPAAR...
1,7,109,sp|Q3MHM5|TBB4B_BOVIN,TUBB4B,49.80,3.3008,MREIVHLQAGQCGNQIGAKFWEVISDEHGIDPTGTYHGDSDLQLER...
2,13,101,sp|Q2T9S0|TBB3_BOVIN,TUBB3,50.40,3.3130,MREIVHIQAGQCGNQIGAKFWEVISDEHGIDPSGNYVGDSDLQLER...
3,62,98,tr|A0A3Q1M0Y8|A0A3Q1M0Y8_BOVIN,CROCC2,169.52,3.1375,MDLEDALGRLEAAEQRSTSLSQVNALLREQLEQLRKAHDRLAEELA...
4,16,91,tr|F6RP72|F6RP72_BOVIN,LOC100295712; tubulin alpha chain,50.85,3.0554,MPHSLLCFQRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPS...
5,80,88,tr|F1MMQ8|F1MMQ8_BOVIN,SPTA1,280.78,3.3721,MESDGPEVLETAEEIQARRQEVLSRYQRFKERVAERGQKLEDSYHY...
6,71,84,tr|F1MKE9|F1MKE9_BOVIN,SPTB,268.25,3.5225,MTSATEFENVGNQPPYSRINARWDGPDDELDNDNSSARLFERSRIK...
7,18,83,tr|A0A452DIL8|A0A452DIL8_BOVIN,TUBB2B,49.94,3.7333,MREIVHIQAGQCGNQIGAKFWEVISDEHGIDPTGSYHGDSDLQLER...
8,27,75,tr|Q58DT9|Q58DT9_BOVIN,ACTA2,45.19,2.7932,MCEEEDSTALVCDNGSGLCKAGFAGDDAPRAVFPSIVGRPRHQGVM...
9,68,72,tr|A0A3Q1MRS3|A0A3Q1MRS3_BOVIN,PLEC,535.22,2.9450,MVAGMFMPLDQLRAIYEVLFREGVMVAKKDRRPRSLHPHVPGVTNL...


In [191]:
raw_ms_data.to_csv("dmtdata_wt_bovine_aaseq.csv",index=False)

In [4]:
def req_uniport(gene):
    #Specify the database to query - we are just quering the general protein database
    url = 'https://www.uniprot.org/uniprot/'

    #entering the parameters we care about
    params= {
        'query':gene,
        'format':'fasta'
    }
    data=urllib.parse.urlencode(params)
    data = data.encode('utf-8')
    req = urllib.request.Request(url,data)
    
    return req


In [8]:
j=req_uniport('aaa33101 1')
decode_uniport(j)

'>sp|P04690|TBB_CHLRE Tubulin beta-1/beta-2 chain OS=Chlamydomonas reinhardtii OX=3055 GN=TUBB1 PE=1 SV=1\nMREIVHIQGGQCGNQIGAKFWEVVSDEHGIDPTGTYHGDSDLQLERINVYFNEATGGRYV\nPRAILMDLEPGTMDSVRSGPYGQIFRPDNFVFGQTGAGNNWAKGHYTEGAELIDSVLDVV\nRKEAESCDCLQGFQVCHSLGGGTGSGMGTLLISKIREEYPDRMMLTFSVVPSPKVSDTVV\nEPYNATLSVHQLVENADECMVLDNEALYDICFRTLKLTTPTFGDLNHLISAVMSGITCCL\nRFPGQLNADLRKLAVNLIPFPRLHFFMVGFTPLTSRGSQQYRALTVPELTQQMWDAKNMM\nCAADPRHGRYLTASALFRGRMSTKEVDEQMLNVQNKNSSYFVEWIPNNVKSSVCDIPPKG\nLKMSATFIGNSTAIQEMFKRVSEQFTAMFRRKAFLHWYTGEGMDEMEFTEAESNMNDLVS\nEYQQYQDASAEEEGEFEGEEEEA\n'